# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [3]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal

import matplotlib.pyplot as plt
import pandas as pd
import pprint


fs = 125  # From the Readme.pdf  (./datasets/troika/Readme.pdf)
acceptable_fs_range = np.array([40, 240]) / 60  # from 40 to 240 in BPM


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of
    the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that
        ref_fls[5] is the
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and
            corresponding reference heart rates.
        confidence_est: a numpy array of confidence estimates
        for each pulse rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and
    returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """ The runner of PulseRateAlgorithm

    This function is used unit test.
    The detailed implementation of the algorithm is delegated
    to the `PulseRateAlgorithm` function


    Args:
        data_fl: (str) filepath to a troika .mat file.
        ref_fl: (str) filepath to a reference file like "REF_XX_TYPEXX.mat"

    Returns:
        errors: The absolute error of pulse rate between
                 estimation and reference in bpm.
        condiden: The metrics introduced to estimate
                  the confidence of the result bpm.
    """

    errors, confidence, _ = PulseRateAlgorithm(data_fl, ref_fl)

    # Return per-estimate mean absolute error and confidence
    # as a 2-tuple of numpy arrays.

    return errors, confidence


#############################################
# The followings are parts of RunPulseRateAlgorithm function
#############################################

def PulseRateAlgorithm(data_fl, ref_fl, algorithm='largest_peak',
                       est_smooth=True, IS_DEBUG=False):
    """ Estimate Pulse Rate and returns thier errors and confidence

    This is the main part of PulseRateAlgorithm.

    Args:
        data_fl: (str) filepath to a troika .mat file.
        ref_fl: (str) filepath to a reference file like "REF_XX_TYPEXX.mat"
        algorithm: (str) algorithm type. The 'largest_peak'
                    or 'distribution_average' are available.
        est_smooth: (bool) If the parameter set True, smoothing is
                    appied to the estimated frequency
        IS_DEBUG: If True is set, this returns debug
                  information as a return value.

    Returns:
        errors: The absolute error of pulse rate between estimation
                and reference in bpm.
        condiden: The metrics introduced to estimate
                  the confidence of the result bpm.
        debug_info: Returns debug info when the IS_DEBUG value is True.
    """

    debug_info = {}

    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # Compute pulse rate estimates and estimation confidence.

    # read ground truth
    ref_bpms = sp.io.loadmat(ref_fl)["BPM0"].reshape(-1)

    # calc total acceleration
    acc = np.sqrt(accx**2 + accy**2 + accz**2)

    # Apply Bandpass Filter to pgg and acc
    ppg_band = BandpassFilter(ppg, acceptable_fs_range, fs)
    acc_band = BandpassFilter(acc, acceptable_fs_range, fs)

    # Calculate FFT with the 8 sec time window and
    # stride 2 sec stride (6 sec overlap)
    ppg_specs, ppg_freqs = Spectrums(ppg_band, fs, NFFT=8*fs, noverlap=6*fs)
    acc_specs, acc_freqs = Spectrums(acc_band, fs, NFFT=8*fs, noverlap=6*fs)

    # Assumes that difference of frequencies is constant.
    freqs = ppg_freqs
    d_freq = freqs[1] - freqs[0]

    # The number of spectrums
    n_specs = ppg_specs.shape[1]

    # Normalize each spectrum
    ppg_specs_nm = [NormalizeByFrequency(
        ppg_specs[:, i], d_freq) for i in range(n_specs)]
    acc_specs_nm = [NormalizeByFrequency(
        acc_specs[:, i], d_freq) for i in range(n_specs)]

    est_bpms = []
    confidences = []

    for i in range(n_specs):
        ppg_spec = ppg_specs_nm[i]
        acc_spec = acc_specs_nm[i]

        if algorithm == 'largest_peak':
            est_bpm, conf = EstimatePulsRateFromLargestPeak(
                ppg_spec, acc_spec, freqs)
        elif algorithm == 'distribution_average':
            est_bpm, conf = EstimatePulsRateFromDistribuionAverage(
                ppg_spec, acc_spec, freqs)
        else:
            raise ValueError(
                "algorithm should be 'largest_peak'" +
                " or 'distribution_average'." +
                f" But got : {algorithm}")

        if est_smooth:
            est_bpm = SmoothResult(est_bpm, est_bpms)

        est_bpms.append(est_bpm)
        confidences.append(conf)

    confidences = np.array(confidences)
    est_bpms = np.array(est_bpms)
    errors = np.abs(ref_bpms - est_bpms)

    if IS_DEBUG:
        debug_info["ppg_specs"] = ppg_specs_nm
        debug_info["acc_specs"] = acc_specs_nm
        debug_info["freqs"] = freqs
        debug_info["est_bpms"] = est_bpms
        debug_info["ref_bpms"] = ref_bpms
        debug_info["errors"] = errors

    return errors, confidences, debug_info


def BandpassFilter(signal, pass_band, fs):
    """Bandpass Filter.

    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>

    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def Spectrums(signal, fs, NFFT, noverlap):
    """Calculate spectrum in a given time window width

    Args:
        signal: (np.array) The input signal
        fs: (number) The sampling rate of <signal>
        NFFT: (number) The number used for the FFT block.
        noverlap: (number) The number of sample that overlap with previous block
    Returns:
        spectrums: (2-D np.array) The list of FFT results.
                   Columns are corresnponding to FFT results.
        freqs: (1-D np.array) The frequences calculated
               by given <fs> and <NFFT> values
    """

    n = len(signal)

    # Each block has (NFFT - noverlap) non overlapped samples,
    step = (NFFT - noverlap)
    n_block = (n - NFFT) // step + 1

    specs = []
    for i in range(n_block):
        w = signal[i * step:i*step+NFFT].copy()

        # the `plt.specgram` apply hanning window on each stride windows
        # w *= np.hanning(len(w))

        fft = np.abs(np.fft.rfft(w))
        specs.append(fft)

    specs = np.stack(specs, axis=1)
    freq = np.fft.rfftfreq(NFFT, d=1/fs)
    return specs, freq


def FrequencyFilter(specs, freqs, freq_range):

    min_freq, max_freq = freq_range

    specs[freqs < min_freq] = 0
    specs[freqs > max_freq] = 0

    return specs


def NormalizeByFrequency(spectrum, d_freq):
    """Normalize signal intencity by frequency

    Args:
        spectrum: (np.array) The input spectrum
        d_freq: The difference of the frequency

    Returns:
        normalized_spectrum: The normalized spectrum
    """

    norm = (spectrum * d_freq).sum()
    return spectrum / norm


def EstimatePulsRateFromLargestPeak(ppg_spec, acc_spec, freqs):
    """ Estimate Puls Rate from the ppg spectrum's lagest peak position

    Args:
        ppg_spec: (np.array) The ppg's spectrum
        acc_spec:(np.array) The accelometer's spectrum
        freqs: (np.array) frequency list that corresponds to ppg_spec and acc_spec
    Returns:
        est_bpm: estimated puls rate in BPM
        conf: The confidence metrics
    """
    ppg_peak = SelectByLargest(ppg_spec, acc_spec)
    est_freq = freqs[ppg_peak]

    conf = EstimatePeakIntensity(ppg_spec, freqs, ppg_peak, 2)
    est_bpm = est_freq * 60.0

    return est_bpm, conf


def EstimatePulsRateFromDistribuionAverage(ppg_spec,
                                           acc_spec,
                                           freqs,
                                           penalty=2):
    """ Estimate Puls Rate from the ppg spectrum distribution

    Args:
        ppg_spec: (np.array) The ppg's spectrum
        acc_spec:(np.array) The accelometer's spectrum
        freqs: (np.array) frequency list that corresponds
               to ppg_spec and acc_spec
        penalty: (numeric) strength how suppress the spectrum
                 when the ppg and acc spectrums overlapped.
    Returns:
        est_bpm: estimated puls rate in BPM
        conf: The confidence metrics
    """
    freq_min, freq_max = acceptable_fs_range

    # Define window area to extract meaningful area
    window = (freqs >= freq_min) & (freqs <= freq_max)

    # Penalize the area of large acc values
    weights = ppg_spec / (1.0 + penalty*acc_spec)

    # Use only windowed region
    freqs = freqs[window]
    weights = weights[window]

    # Normalize for the window area
    weights = weights / weights.sum()

    # Calculate mean freqeuncy as a estimated puls rate
    freq_mean = (weights * freqs).sum()

    # Confidence is defined by 1/(1+std) as a approximation
    # of the inverse of standard deviation
    freq_var = (((freqs - freq_mean)**2) * weights).sum()
    freq_stddev = np.sqrt(freq_var / len(freqs))
    conf = 1/(1.0+freq_stddev)

    est_bpm = 60*freq_mean

    return est_bpm, conf


def EstimatePeakIntensity(spectrum, freqs, peak, peak_width):
    """ Calculate intensity of peak

    As a confidency, the intensity of peak is estimated
    by the sum of the spectrum around the peak.

    Args:
        spectrum : (np.array) The ppg's spectrum.
        freqs: (np.array) frequency list that corresponds to spectrum.
        peak_width: (int) the width of frequency space that include intensity.
        The spectrum between `peak-peak_width` and `peak+peak_width` are
        included in the summation
    Returns:
        confidence: (numeric) The ratio between peak intensity
                    and total sum of spectrum.

    """

    n = len(freqs)
    peak_area = np.arange(peak-peak_width, peak+peak_width+1)
    peak_area = peak_area[np.where((peak_area >= 0) & (peak_area < n))[0]]

    return spectrum[peak_area].sum() / spectrum.sum()


def SelectByLargest(ppg_spec, acc_spec):
    """ Returns estimated pulse rate that takes into account
        the effect of acceleration.
    """

    ppg_cands = np.argsort(ppg_spec)[::-1]
    acc_peaks, _ = scipy.signal.find_peaks(acc_spec, height=0.5)

    for pk in ppg_cands[:3]:
        if pk not in acc_peaks:
            return pk

    # if upto the 3d largest position is overlaped with acc,
    # returns the lagest peak
    return ppg_cands[0]


def SmoothResult(cur_est, est_history):
    """ Apply smoothing the estimated pulse rate

    Args:
        cur_est : (numeric) The current estimation before the smoothing.
        est_history: (list) The list of previous result.

    Returns:
        smoothed_estimation: (numeric) The ratio between peak intensity
        and total sum of spectrum.
    """

    if len(est_history) < 3:
        return cur_est
    else:
        return (
            0.4*cur_est + 0.3*est_history[-1] +
            0.2*est_history[-2] +
            0.1*est_history[-3])
